In [137]:
import numpy as np
import pandas as pd
from numpy.random import shuffle
from numpy.random import Generator as gen 
from numpy.random import PCG64 as pcg


In [138]:
np.set_printoptions(suppress=True, linewidth=100, precision=2)

# Importing data

In [139]:
raw_data = np.genfromtxt('data/loan-data.csv', delimiter=";", skip_header = 1, autostrip = True, encoding = "cp855")
raw_data

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

# Checking data

In [140]:
np.isnan(raw_data).sum()

88005

In [141]:
temporary_mean = np.nanmean(raw_data, axis=0)

temporary_fill= np.nanmax(raw_data) + 1


temporary_stats = np.array([np.nanmin(raw_data, axis=0), temporary_mean, np.nanmax(raw_data, axis=0)])

temporary_stats

/tmp/ipykernel_3906871/2718681720.py:1: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data, axis=0)
/tmp/ipykernel_3906871/2718681720.py:6: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data, axis=0), temporary_mean, np.nanmax(raw_data, axis=0)])


array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

# Splitting 

In [142]:
columns_string = np.argwhere(np.isnan(temporary_mean)).squeeze()
columns_string

array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [143]:
columns_numeric = np.argwhere(np.isnan(temporary_mean)==False).squeeze()
columns_numeric

array([ 0,  2,  4,  6,  7, 13])

# Re-importing data seprately

In [144]:
loan_string_data = np.genfromtxt('data/loan-data.csv', delimiter=";", skip_header=1, 
                            usecols=columns_string, encoding="cp855", dtype=np.str_)
loan_string_data

array([['May-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', ' 36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [145]:
loan_numeric_data = np.genfromtxt('data/loan-data.csv', delimiter=";", skip_header=1, 
                            usecols=columns_numeric, encoding="cp855", filling_values=temporary_fill)
loan_numeric_data

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

# Rename columns

In [146]:
full_header = np.genfromtxt('data/loan-data.csv', delimiter=";", skip_footer=raw_data.shape[0], 
                            encoding="cp855", dtype=np.str_)
full_header

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [147]:
string_headers, numeric_headers = full_header[columns_string], full_header[columns_numeric]
    

In [148]:
string_headers

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [149]:
numeric_headers

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

# Creating checkpoint

In [150]:
def check_point(filename, checkpoint_header, checkpoint_data):
    np.savez(file=filename, header=checkpoint_header, data=checkpoint_data)
    variable_checkpoint = np.load("{}.npz".format(filename))
    return (variable_checkpoint)

In [151]:
check_point_test = check_point('loan_string_data', string_headers, loan_string_data)

In [152]:
check_point_test['data']

array([['May-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', ' 36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', ' 36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', ' 36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

# Manipulation String Columns

In [153]:
string_headers

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [154]:
string_headers[0] = 'issue_date'

### Issue data

In [155]:
np.unique(loan_string_data[:,0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

In [156]:
loan_string_data[:,0] = np.chararray.strip(loan_string_data[:, 0], '-15')
loan_string_data[:,0]

array(['May', '', 'Sep', ..., 'Jun', 'Apr', 'Dec'], dtype='<U69')

In [157]:
months = np.array(['', 'Jan', 'Feb', 'Mar', "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])

In [158]:
for i in range(13):
    loan_string_data[:,0] = np.where(loan_string_data[:,0]==months[i],i, loan_string_data[:,0])
    
loan_string_data[:, 0]    

array(['5', '0', '9', ..., '6', '4', '12'], dtype='<U69')

### Loan Status

In [159]:
np.unique(loan_string_data[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [160]:
bad_status = np.array(['Charged Off', "Default", "Late (31-120 days)"])

In [161]:
loan_string_data[:,1] = np.where(np.isin(loan_string_data[:,1], bad_status), 0, 1)

In [162]:
np.unique(loan_string_data[:, 1])

array(['0', '1'], dtype='<U69')

### Terms 

In [163]:
string_headers

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [164]:
string_headers[2] = "terms months"

In [165]:
np.unique(loan_string_data[:,2])

array(['', ' 36 months', ' 60 months'], dtype='<U69')

In [166]:
loan_string_data[:,2] = np.chararray.strip(loan_string_data[:,2], " months ")
loan_string_data[:, 2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [167]:
loan_string_data[:,2] = np.where(loan_string_data[:,2]=='', '60', loan_string_data[:,2])

In [168]:
loan_string_data[:, 2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

### Grade and Sub-Grade

In [169]:
string_headers

array(['issue_date', 'loan_status', 'terms months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [170]:
np.unique(loan_string_data[:, 3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

In [171]:
np.unique(loan_string_data[:, 4], return_counts=True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([514, 285, 278, 239, 323, 502, 509, 517, 530, 553, 494, 629, 567, 586, 564, 423, 391, 267,
        250, 255, 223, 235, 162, 171, 139, 114,  94,  52,  34,  43,  16,  19,  10,   3,   7,   2]))

 ### Filling Subgrade

In [172]:
for i in np.unique(loan_string_data[:, 3])[1:]:
    loan_string_data[:,4] = np.where((loan_string_data[:, 4]=='') & (loan_string_data[:,3]==i), i + "5",
                                    loan_string_data[:, 4])
np.unique(loan_string_data[:,4], return_counts=True) 

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5]))

In [173]:
loan_string_data[:,4] = np.where(loan_string_data[:, 4]=='', "H1",
                                    loan_string_data[:, 4])
np.unique(loan_string_data[:,4], return_counts=True) 

(array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
        'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
        'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69'),
 array([285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267, 250,
        255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5,   9]))

### Removing grade columns

In [174]:
loan_string_data = np.delete(loan_string_data, 3, axis=1)

In [175]:
loan_string_data[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

In [176]:
string_headers = np.delete(string_headers, 3)
string_headers[3]

'sub_grade'

### Converting sub-grade to number

In [177]:
np.unique(loan_string_data[:, 3], return_counts=True)

(array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
        'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
        'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69'),
 array([285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267, 250,
        255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5,   9]))

In [178]:
keys = list(np.unique(loan_string_data[:, 3]))
values = list(range(1, np.unique(loan_string_data[:, 3]).shape[0] + 1))
dict_sub_grade = dict(zip(keys, values))

In [179]:
dict_sub_grade

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'B1': 6,
 'B2': 7,
 'B3': 8,
 'B4': 9,
 'B5': 10,
 'C1': 11,
 'C2': 12,
 'C3': 13,
 'C4': 14,
 'C5': 15,
 'D1': 16,
 'D2': 17,
 'D3': 18,
 'D4': 19,
 'D5': 20,
 'E1': 21,
 'E2': 22,
 'E3': 23,
 'E4': 24,
 'E5': 25,
 'F1': 26,
 'F2': 27,
 'F3': 28,
 'F4': 29,
 'F5': 30,
 'G1': 31,
 'G2': 32,
 'G3': 33,
 'G4': 34,
 'G5': 35,
 'H1': 36}

In [180]:
for i in keys:
    loan_string_data[:,3] = np.where(loan_string_data[:, 3]==i, dict_sub_grade[i], loan_string_data[:,3])

loan_string_data[:, 3]    

array(['13', '5', '10', ..., '5', '17', '4'], dtype='<U69')

### Verification Status

In [181]:
string_headers

array(['issue_date', 'loan_status', 'terms months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [184]:
np.unique(loan_string_data[:, 4])

array(['0', '1'], dtype='<U69')

In [183]:
bad_status_verified = np.array(['', 'Not Verified'])

loan_string_data[:,4] = np.where(np.isin(loan_string_data[:,4] , bad_status_verified), 0, 1)
loan_string_data[:,4]

array(['1', '1', '1', ..., '1', '1', '0'], dtype='<U69')

### Url 

In [185]:
string_headers

array(['issue_date', 'loan_status', 'terms months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [186]:
loan_string_data[:, 5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], dtype='<U69')

In [187]:
loan_string_data[:,5] = np.chararray.strip(loan_string_data[:, 5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')

In [188]:
loan_string_data[:, 5]

array(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'], dtype='<U69')

In [194]:
loan_numeric_data[:,0]

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

In [197]:
np.array_equal(loan_string_data[:, 5].astype(dtype=np.int32), loan_numeric_data[:, 0].astype(dtype=np.int32))

True

In [199]:
np.delete(loan_string_data, 5, axis=1)

array([['5', '1', '36', '13', '1', 'CA'],
       ['0', '1', '36', '5', '1', 'NY'],
       ['9', '1', '36', '10', '1', 'PA'],
       ...,
       ['6', '1', '36', '5', '1', 'CA'],
       ['4', '1', '36', '17', '1', 'OH'],
       ['12', '1', '36', '4', '0', 'IL']], dtype='<U69')

In [200]:
np.delete(string_headers, 5)

array(['issue_date', 'loan_status', 'terms months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')